# Import Dependencies 

In [16]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# Download Data

In [17]:
mnist = input_data.read_data_sets("data/MNIST/", one_hot=True)

Extracting data/MNIST/train-images-idx3-ubyte.gz
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz


## Note: One Hot Encoding

#### One is hot, the rest are cold

0 = [1,0,0,0,0,0,0,0,0,0]

1 = [0,1,0,0,0,0,0,0,0,0]

2 = [0,0,1,0,0,0,0,0,0,0]

# Hidden Layers

In [18]:
n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500

# How many classes to label
n_classes = 10

# Batch size to feed the NN
batch_size = 100

# Input Placeholders

In [19]:
# X is a tensor, which is a generalization of vectors and matrices. This tensor object will store an image
# data type, shape = [height, width]
# width is equal to 784, because we are reducing the dimension of the image to 1D, while still containing the same info.
x = tf.placeholder('float', [None, 784])

# labels
y = tf.placeholder('float')

# NN Model

In [70]:
def neural_network_model(data):
    
    ##### CREATE HIDDEN LAYERS

    # an edge between two nodes will have a weight
    # biases are associated with each node, except input nodes
    hidden_1_layer = {'weights':tf.Variable(tf.random_normal([784, n_nodes_hl1])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}
    
    hidden_2_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl2]))}
    
    hidden_3_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                      'biases':tf.Variable(tf.random_normal([n_nodes_hl3]))}
    
    output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                    'biases':tf.Variable(tf.random_normal([n_classes]))}
    
    ##### w*x + bias , activation function
    
    l1 = tf.matmul(data, hidden_1_layer['weights']) +  hidden_1_layer['biases']
    l1 = tf.nn.relu(l1)
    
    l2 = tf.matmul(l1, hidden_2_layer['weights']) + hidden_2_layer['biases']
    l2 = tf.nn.relu(l2)
    
    l3 = tf.matmul(l2, hidden_3_layer['weights']) + hidden_3_layer['biases']
    l3 = tf.nn.relu(l3)
    
    output = tf.matmul(l3, output_layer['weights']) + output_layer['biases']
    
    return output

# Train & Predict The Model

In [80]:
def train_neural_network(x):
    
    # measures how wrong the prediction is 
    prediction = neural_network_model(x)
    
    # reduce mean: takes the average of the passed argument. 
    # softmax: a cost/error function that normalizes the inputs so that the sum(input) = 1, outputs probabilities
    # Cross Entropy: the distance between two probability distributions, vectors.
    # with logits: unscaled input that we want to scale so that its sum is equal to 1
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))
    
    # calculates an exponential moving average of the gradient. Learning rate isnt constant
    # alternative to SGD, where learning rate is constant
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    # cycles of forward feed and back propagation
    epochs = 6
    
    # to run TensorFlow Ops
    with tf.Session() as session:
        
        # run takes a list of 'operations'
        # before we can use variables resulted from TF Ops, we need to initialize them. 
        session.run(tf.global_variables_initializer())
        
        for epoch in range(epochs):
            epoch_loss = 0
            
            # iterate over the number of image groups which are created from the MNIST training set
            for _ in range(int(mnist.train.num_examples/batch_size)):
                
                # mnist.train.next_batch iterates through the group of images and stores them in 'x' and label 'y'
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                
                # runs a one step operations in variables optimizer and cost
                # arguments are stored in feed_dictionary
                # store cost in C
                _, c = session.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                
                # sum all the loss for one epoch for the complete training set
                epoch_loss += c
                
            print('Epoch', epoch, 'complete out of', epochs, 'loss:', epoch_loss)
        
        # argmax returns indices of the maximum values 
        # if the two indices in prediction and y are the same, then the prediction is correct, otherwise, incorrect
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        
        # outputs the average accuracy of the test set
        # tf.cast will convert the boolean 'correct' variable into a float
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        
        # will run the two lines resulted from the variables 'correct' & 'accuracy'
        print('Accuracy:',accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))
                

In [81]:
train_neural_network(x)

Epoch 0 complete out of 6 loss: 1673399.0471191406
Epoch 1 complete out of 6 loss: 396514.19585227966
Epoch 2 complete out of 6 loss: 213293.1284813881
Epoch 3 complete out of 6 loss: 125084.58017386455
Epoch 4 complete out of 6 loss: 77364.55336558819
Epoch 5 complete out of 6 loss: 51161.409061570674
Accuracy: 0.9367
